In [1]:
import requests


def generate_answer_with_ollama(query, context=' ', model="llama3:latest", url="http://localhost:11434/api/generate"):
    input_text = f"Question: {query}\nContext: {context}"
    payload = {
        'prompt': input_text,
        'model': model,  # Add the model parameter here
        'max_tokens': 50  # Set the maximum number of tokens to generate
    }

    response = requests.post(url, json=payload)
    content_type = response.headers.get('Content-Type')    # if response.status_code == 200:
    print("Content-Type:", content_type)

    if 'application/x-ndjson' in content_type:
        try:
            # Process NDJSON response
            responses = response.content.decode('utf-8').splitlines()
            json_responses = [requests.models.complexjson.loads(line) for line in responses]
            responses = [item["response"] for item in json_responses]
            text = ''.join(responses)
            # print("NDJSON parsed successfully:", text)

            # Extract the answer from the first JSON object
            return text
        except Exception as e:
            print("Failed to decode NDJSON response:", e)
            print("Response content:", response.content)
            raise
    else:
        print("Non-NDJSON response received")
        print("Response content:", response.content)
        raise Exception(f"Unexpected content type: {content_type}")
    
    
# generate_answer_with_ollama('Seperate the following text into meaning--complete sentences:\n\n', context="""Why then? Call a fucking pharmacist who can tell you what the consequences are, IF ANY. That's their job. Or pay the money to talk to a doctor on a holiday. That is what you do. Don't come on here and post the question to a bunch of people who are unqualified to answer. Get off the fucking computer and call someone. Jesus Christ.""")

In [2]:
import os
path = "C:/Users/L/.convokit/downloads/"
os.environ['http_proxy'] = 'http://localhost:7890'
os.environ['https_proxy'] = 'http://localhost:7890'
directories = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
print(directories)

['conversations-gone-awry-cmv-corpus', 'subreddit-ADD', 'subreddit-ADHD', 'subreddit-AmericanPolitics', 'subreddit-Cornell', 'subreddit-NSFW_Social', 'subreddit-POLUG3']


In [40]:
import pandas as pd
from convokit import Corpus, download


# corpus = Corpus(filename="C:/Users/L/.convokit/downloads/conversations-gone-awry-cmv-corpus")
corpus = Corpus(filename="C:/Users/L/.convokit/downloads/subreddit-AmericanPolitics")
corpus.print_summary_stats()

utt_df = corpus.get_utterances_dataframe().drop(columns=['vectors'])
convo_df = corpus.get_conversations_dataframe().drop(columns=['vectors'])
speaker_df = corpus.get_speakers_dataframe().drop(columns=['vectors'])


Number of Speakers: 5592
Number of Utterances: 66169
Number of Conversations: 29492


In [39]:
def print_overview(filename):
    corpus = Corpus(filename=filename)
    corpus.print_summary_stats()
    
    utt_df = corpus.get_utterances_dataframe().drop(columns=['vectors'])
    convo_df = corpus.get_conversations_dataframe().drop(columns=['vectors'])
    speaker_df = corpus.get_speakers_dataframe().drop(columns=['vectors'])
    print("UttDf attributes:", list(utt_df.columns),'\n')
    """gilded: 内容的镀金状态
    gildings: 内容的镀金信息
    stickied: 内容的置顶状态
    author_flair_text: 作者的标志"""
    
    print("ConvDf attributes:", list(convo_df.columns),'\n')
    print("SpeakerDf attributes:", list(speaker_df.columns),'\n')

    print(convo_df.head(2))
    print(convo_df.shape)
    print(utt_df.head(2))
    print(utt_df.shape)
    print(speaker_df.head(2))
    print(speaker_df.shape)
    
print_overview("C:/Users/L/.convokit/downloads/conversations-gone-awry-cmv-corpus")
print("#################")
print_overview("C:/Users/L/.convokit/downloads/subreddit-AmericanPolitics")

Number of Speakers: 9548
Number of Utterances: 42964
Number of Conversations: 6842
UttDf attributes: ['timestamp', 'text', 'speaker', 'reply_to', 'conversation_id', 'meta.score', 'meta.top_level_comment', 'meta.retrieved_on', 'meta.gilded', 'meta.gildings', 'meta.subreddit', 'meta.stickied', 'meta.permalink', 'meta.author_flair_text', 'meta.parsed'] 

ConvDf attributes: ['meta.pair_id', 'meta.has_removed_comment', 'meta.split', 'meta.summary_meta'] 

SpeakerDf attributes: ['meta.num_posts', 'meta.num_comments'] 

        meta.pair_id meta.has_removed_comment meta.split meta.summary_meta
id                                                                        
cue8y0b      cue8uxd                     True      train                []
cue8uxd      cue8y0b                    False      train                []
(6842, 4)
          timestamp                                               text  \
id                                                                       
cue8y0b  1440446000  (O

In [41]:
print(utt_df.loc['cksg6','text'])
type(utt_df.loc['cksg6','text'])

str

In [42]:
unsilent_df = utt_df[utt_df['text']!=''].copy()
print(unsilent_df.shape)

(37448, 14)


In [43]:
new_corpus = Corpus.from_pandas(utterances_df=unsilent_df, speakers_df=speaker_df, conversations_df=convo_df)
new_corpus.print_summary_stats()

ID column is not present in utterances dataframe, generated ID column from dataframe index...
ID column is not present in conversations dataframe, generated ID column from dataframe index...
ID column is not present in speakers dataframe, generated ID column from dataframe index...


37448it [00:01, 24256.92it/s]


Number of Speakers: 3717
Number of Utterances: 37448
Number of Conversations: 12769


In [48]:
speaker_df=new_corpus.get_speakers_dataframe()
convo_df=new_corpus.get_conversations_dataframe()
utt_df=new_corpus.get_utterances_dataframe()
print(convo_df.sample(n=5))
print(utt_df.sample(n=5))

       vectors                                         meta.title  \
id                                                                  
6qzz4m      []  BREAKING NEWS: CONGRESS DEMANDS THE ARREST AND...   
1sju0c      []      Can You Pass The Terrorism Quiz (Great Info.)   
xarby       []              The New York Pirate Party's subreddit   
20kja9      []  Mr Obama, you are the biggest fraud that has e...   
26cmdm      []  “Why Our Political System’s Screwed, in One Ve...   

       meta.num_comments                             meta.domain  \
id                                                                 
6qzz4m                 2                                youtu.be   
1sju0c                 3  detailedpoliticalquizzes.wordpress.com   
xarby                  0                              reddit.com   
20kja9                 1                             youtube.com   
26cmdm                 6              againstcronycapitalism.org   

       meta.timestamp    meta.subreddit

In [52]:
print(convo_df[convo_df['meta.gilded'] < 0])

      vectors                                         meta.title  \
id                                                                 
re98x      []                   New subreddit:    /r/R3VOLUTION/   
fekv7      []                                  Break up the USA?   
h6el1      []  Hey, I promise to vote for Ron Paul if you wil...   
hbaj0      []  Is there a quiz to determine which party I am ...   
g5183      []         I don't hate the rest of the world, but...   
...       ...                                                ...   
z30li      []  Does Rand Paul Have the Future? | The American...   
z4doa      []                              America's Choice-2012   
z4vbr      []    Romney's "RNC Power Grab": What Really Happened   
z5kfq      []  John Stossel TV Show Will Interview Four Minor...   
z5rys      []                       Eastwood and the Empty Chair   

      meta.num_comments                  meta.domain meta.timestamp  \
id                                          

In [46]:
popular_convo_df = convo_df[convo_df['meta.num_comments'] > 10]
print(popular_convo_df.head(5))


      vectors                                         meta.title  \
id                                                                 
cb6pf      []  I called my congressman about Israel's attack ...   
fekv7      []                                  Break up the USA?   
hbaj0      []  Is there a quiz to determine which party I am ...   
79j92      []  Why do "Good Christians"  espouse all "Ignoran...   
efbbk      []  Americans are asleep at the wheel while our go...   

      meta.num_comments            meta.domain meta.timestamp  \
id                                                              
cb6pf                12  self.AmericanPolitics     1275602113   
fekv7                21  self.AmericanPolitics     1296751065   
hbaj0                12  self.AmericanPolitics     1305389222   
79j92                13            youtube.com     1225074213   
efbbk                11  self.AmericanPolitics     1291335247   

         meta.subreddit meta.gilded meta.gildings meta.stickied  \


In [29]:

print(convo_df.loc['cue8y0b'])
print(convo_df.loc[:,'pair_id'])


KeyError: 'cue8y0b'

In [21]:
utt_df.head(20)
# utt_df.loc['cue8y0b']

,timestamp,text,speaker,reply_to,conversation_id,meta.score,meta.top_level_comment,meta.retrieved_on,meta.gilded,meta.gildings,meta.subreddit,meta.stickied,meta.permalink,meta.author_flair_text,meta.parsed,id
id,,,,,,,,,,,,,,,,
cue8y0b,1440446000,"(Okay, I've seen this view come up a few times...",RustyRook,None,cue8y0b,17,cue8y0b,1441446236,0,None,changemyview,False,,,"[{'rt': 5, 'toks': [{'tok': '(', 'tag': '-LRB-...",cue8y0b
cuec5fs,1440450798,It's not just black and white America though. ...,Canada_is_gay,cue8y0b,cue8y0b,4,cue8y0b,1441447812,0,None,changemyview,False,,,"[{'rt': 1, 'toks': [{'tok': 'It', 'tag': 'PRP'...",cuec5fs
cuect48,1440451823,Abstract reasoning is a skill that can be nurt...,aguafiestas,cuec5fs,cue8y0b,3,cue8y0b,1441448123,0,None,changemyview,False,,7∆,"[{'rt': 2, 'toks': [{'tok': 'Abstract', 'tag':...",cuect48
cuedf8c,1440452797,Can we agree that genes account for about 50% ...,ScholarlyVirtue,cuect48,cue8y0b,3,cue8y0b,1441448412,0,None,changemyview,False,,2∆,"[{'rt': 2, 'toks': [{'tok': 'Can', 'tag': 'MD'...",cuedf8c
cuedywn,1440453690,Twin studies studies suggest that about 80 per...,Perpetualjoke,cuedf8c,cue8y0b,7,cue8y0b,1441448670,0,None,changemyview,False,,,"[{'rt': 3, 'toks': [{'tok': 'Twin', 'tag': 'JJ...",cuedywn
czb942p,1453734180,"Sorry Perpetualjoke, your comment has been rem...",garnteller,cuedywn,cue8y0b,1,cue8y0b,1454717903,0,None,changemyview,False,,148∆,"[{'rt': 7, 'toks': [{'tok': 'Sorry', 'tag': 'N...",czb942p
czbbocu,1453738298,Are you kidding me?You decided to remove it af...,Perpetualjoke,czb942p,cue8y0b,1,cue8y0b,1454719112,0,None,changemyview,False,,,"[{'rt': 2, 'toks': [{'tok': 'Are', 'tag': 'VBP...",czbbocu
czbdh6q,1453740934,You decided to edit it after 5 months. Why wo...,garnteller,czbbocu,cue8y0b,1,cue8y0b,1454719971,0,None,changemyview,False,,148∆,"[{'rt': 1, 'toks': [{'tok': 'You', 'tag': 'PRP...",czbdh6q
czbe470,1453741858,I just can't believe you are browsing 5 month ...,Perpetualjoke,czbdh6q,cue8y0b,1,cue8y0b,1454720272,0,None,changemyview,False,,,"[{'rt': 16, 'toks': [{'tok': 'I', 'tag': 'PRP'...",czbe470


In [19]:
new_corpus = Corpus.from_pandas(utterances_df=utt_df, speakers_df=speaker_df, conversations_df=convo_df) 
new_corpus.print_summary_stats()

42964it [00:01, 22703.65it/s]


Number of Speakers: 9548
Number of Utterances: 42964
Number of Conversations: 6842


In [7]:
# new_corpus.get_conversation('cue8y0b')
print(new_corpus.get_object('conversation','cue8y0b'))
print(new_corpus.get_object('utterance','cue8y0b'))


Conversation('id': 'cue8y0b', 'utterances': ['cue8y0b', 'cuec5fs', 'cuect48', 'cuedf8c', 'cuedywn', 'czb942p', 'czbbocu', 'czbdh6q', 'czbe470', 'czbe8el'], 'meta': ConvoKitMeta({'pair_id': 'cue8uxd', 'has_removed_comment': True, 'split': 'train', 'summary_meta': []}))
Utterance(id: 'cue8y0b', conversation_id: cue8y0b, reply-to: None, speaker: Speaker(id: 'RustyRook', vectors: [], meta: {}), timestamp: 1440446000, text: "(Okay, I've seen this view come up a few times before and I've always been unsuccessful in convincing people about why they're wrong. However, it seems that your view is based on studies, so maybe you'll respond well to evidence /u/Carlosriccy.)\n\nI cannot dispute the fact that there is a measurable IQ gap b/w white students and black students in the US. However, it is most certainly not because of genetics. Almost all of it can be attributed to social causes - black students are often in schools that aren't adequately funded, poverty and its corollary effects, etc. \n

In [9]:
for convo in list(corpus.iter_conversations())[:3]:
    utterance_ids = convo.get_utterance_ids()
    print(utterance_ids)
    for utt_id in utterance_ids:
        utt = corpus.get_utterance(utt_id)
        print(utt.text, "[SEP]")

['cksg6', 'c0ta4bv', 'c0tafws']
 [SEP]
when they can no longer use all the money for smoking cessation programs, smoking prevention, and cancer treatment research.  oh, they don't use much of it for that to begin with, well then it is a sin tax, and is only meant to punish people who like something that others don't.  i think the tax on tithes should be linked to the tax on alcohol and tobacco. [SEP]
This is one of the few areas where I agree with libertarians....sin taxes are silly. [SEP]
['ckt8r']
 [SEP]
['cl1ih']
 [SEP]


In [30]:
popular_convo_df = convo_df[convo_df['meta.num_comments'] > 10]
print(popular_convo_df.head(5))
print(popular_convo_df.shape)


KeyError: 'meta.num_comments'

In [ ]:
for index, convo in popular_convo_df.iterrows():
    conversation = convo['conversation']  # 获取对话对象
    print(conversation, convo)
    convo = convo[0] #panda series to 
    
    utterance_ids = convo.get_utterance_ids()
    print(utterance_ids)
    for utt_id in utterance_ids:
        utt = corpus.get_utterance(utt_id)
        print(utt.text, "[SEP]")

In [29]:
# 获取讲话者属性
speaker_attrs = set()
for speaker in corpus.iter_speakers():
    speaker_attrs.update(speaker.meta.keys())
    break  # 仅检查第一个讲话者

print("Speaker attributes:", speaker_attrs)

# 获取对话属性
convo_attrs = set()
for convo in corpus.iter_conversations():
    convo_attrs.update(convo.meta.keys())
    break  # 仅检查第一个对话

print("Conversation attributes:", convo_attrs)

# 获取话语属性
utterance_attrs = set()
for utterance in corpus.iter_utterances():
    utterance_attrs.update(utterance.meta.keys())
    break  # 仅检查第一个话语

print("Utterance attributes:", utterance_attrs)

Speaker attributes: set()
Conversation attributes: {'title', 'timestamp', 'gildings', 'author_flair_text', 'gilded', 'stickied', 'subreddit', 'num_comments', 'domain'}
Utterance attributes: {'subreddit', 'permalink', 'gildings', 'top_level_comment', 'author_flair_text', 'gilded', 'stickied', 'score', 'retrieved_on'}


In [25]:
# 为每个对话添加一个唯一的 id
for idx, convo in enumerate(corpus.iter_conversations()):
    convo.meta['id'] = f'convo_{idx}'

# 验证
for convo in corpus.iter_conversations():
    print(convo.meta['id'])
    break  # 打印第一个对话的 id
    
# 为每个话语添加一个唯一的 id
for idx, utterance in enumerate(corpus.iter_utterances()):
    utterance.meta['id'] = f'utt_{idx}'

# 验证
for utterance in corpus.iter_utterances():
    print(utterance.meta['id'])
    break  # 打印第一个话语的 id

# 为每个讲话者添加一个唯一的 id
for idx, speaker in enumerate(corpus.iter_speakers()):
    speaker.meta['id'] = f'speaker_{idx}'

# 验证
for speaker in corpus.iter_speakers():
    print(speaker.meta['id'])
    break  # 打印第一个讲话者的 id


convo_0
utt_0
speaker_0


In [33]:
for speaker in corpus.iter_speakers():
    print(speaker.meta)
    # print(speaker.meta["conversations"]).items()
    break

ConvoKitMeta({})


In [18]:
full_df = corpus.get_full_attribute_table(speaker_convo_attrs)


KeyError: "None of ['id'] are in the columns"

In [22]:
utt_df.head(20)
# print(convo_df.head(5))
# print(speaker_df.head(5))


,timestamp,text,speaker,reply_to,conversation_id,meta.score,meta.top_level_comment,meta.retrieved_on,meta.gilded,meta.gildings,meta.subreddit,meta.stickied,meta.permalink,meta.author_flair_text
id,,,,,,,,,,,,,,
cksg6,1277954798,,jack_alexander,None,cksg6,3,None,1522932016,0,None,AmericanPolitics,False,/r/AmericanPolitics/comments/cksg6/watercooler...,
ckt8r,1277959831,,jack_alexander,None,ckt8r,6,None,1522932034,0,None,AmericanPolitics,False,/r/AmericanPolitics/comments/ckt8r/another_war...,
cl1ih,1278011224,,quakerorts,None,cl1ih,5,None,1522932199,0,None,AmericanPolitics,False,/r/AmericanPolitics/comments/cl1ih/out_of_afgh...,
cl3qt,1278023178,,Designthing,None,cl3qt,3,None,1522932244,0,None,AmericanPolitics,False,/r/AmericanPolitics/comments/cl3qt/ej_dionne_j...,
cl4a2,1278026741,,[deleted],None,cl4a2,13,None,1522932255,0,None,AmericanPolitics,False,/r/AmericanPolitics/comments/cl4a2/238_preside...,
cl61l,1278039750,,[deleted],None,cl61l,2,None,1522932292,0,None,AmericanPolitics,False,/r/AmericanPolitics/comments/cl61l/james_k_gal...,
cl76a,1278047999,,jack_alexander,None,cl76a,5,None,1522932314,0,None,AmericanPolitics,False,/r/AmericanPolitics/comments/cl76a/pot_versus_...,
clcn3,1278085927,,[deleted],None,clcn3,2,None,1522932425,0,None,AmericanPolitics,False,/r/AmericanPolitics/comments/clcn3/okgov_candi...,
cle3f,1278093550,,therealjerrystaute,None,cle3f,1,None,1522932455,0,None,AmericanPolitics,False,/r/AmericanPolitics/comments/cle3f/the_three_b...,


In [21]:
convo_df.head(10)

,meta.title,meta.num_comments,meta.domain,meta.timestamp,meta.subreddit,meta.gilded,meta.gildings,meta.stickied,meta.author_flair_text
id,,,,,,,,,
cksg6,Watercooler – Cigarette Tax: How High Is Too H...,2,seminal.firedoglake.com,1277954798,AmericanPolitics,0,None,False,
ckt8r,"Another War Where We Asked ""Why Are We Here?"" ...",0,newstalgia.crooksandliars.com,1277959831,AmericanPolitics,0,None,False,
cl1ih,‘Out of Afghanistan Caucus’ pledges to keep vo...,0,rawstory.com,1278011224,AmericanPolitics,0,None,False,
cl3qt,E.J. Dionne Jr. tees off on Democrats whining ...,0,washingtonpost.com,1278023178,AmericanPolitics,0,None,False,
cl4a2,238 presidential scholars: President George W....,5,rawstory.com,1278026741,AmericanPolitics,0,None,False,
cl61l,James K. Galbraith - Why the fiscal commission...,0,newdeal20.org,1278039750,AmericanPolitics,0,None,False,
cl76a,Pot Versus Alcohol: Experts Say Booze Is the B...,0,alternet.org,1278047999,AmericanPolitics,0,None,False,
clcn3,"OK-GOV: Candidates take on prison crowding, wa...",0,tulsaworld.com,1278085927,AmericanPolitics,0,None,False,
cle3f,The three biggest lies about the economy: The ...,0,marketwatch.com,1278093550,AmericanPolitics,0,None,False,


In [20]:
speaker_df.head(5)

""
id
jack_alexander
quakerorts
Designthing
[deleted]
therealjerrystaute


In [ ]:
full_df.head(5)

In [ ]:


# 计算每个用户参与的对话数
user_conversation_counts = convo_df['user_id'].value_counts()

# 过滤出参与对话数超过三个的用户
active_users = user_conversation_counts[user_conversation_counts > 3].index

# 过滤出活跃度最大的几个用户
top_active_users = speaker_df[speaker_df['user_id'].isin(active_users)].nlargest(10, 'metadata')

# 显示结果
print("参与对话数超过三个的用户：")
print(speaker_df[speaker_df['user_id'].isin(active_users)])

In [7]:
print(corpus)
# Extract data from the corpus
conversations = list(corpus.iter_conversations())
utterances = list(corpus.iter_utterances())
users = list(corpus.iter_users())

# Convert the extracted data into pandas DataFrames
convo_df = pd.DataFrame([convo.meta for convo in conversations])
utt_df = pd.DataFrame([utt.meta for utt in utterances])
speaker_df = pd.DataFrame([user.meta for user in users])

# Display the DataFrames
print(convo_df.head())
print(utt_df.head())
print(speaker_df.head())

AttributeError: 'Corpus' object has no attribute 'iter_users'